In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import mediapipe as mp

In [2]:
mp_holistic=mp.solutions.holistic
mp_drawing=mp.solutions.drawing_utils

In [3]:

def holistic_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [5]:
"""cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
 while cap.isOpened():
    ret,frame=cap.read()
    
    #making detections
    image,results=holistic_detection(frame,holistic)
   # print(results)
    draw_styled_landmarks(image, results)
    cv2.imshow('OpenCv Feed',image)
    if cv2.waitKey(10)& 0xFF==ord('q'):
        break
 cap.release
 cv2.destroyAllWindows() """   

"cap=cv2.VideoCapture(0)\nwith mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:\n while cap.isOpened():\n    ret,frame=cap.read()\n    \n    #making detections\n    image,results=holistic_detection(frame,holistic)\n   # print(results)\n    draw_styled_landmarks(image, results)\n    cv2.imshow('OpenCv Feed',image)\n    if cv2.waitKey(10)& 0xFF==ord('q'):\n        break\n cap.release\n cv2.destroyAllWindows() "

In [54]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
#result_test = extract_keypoints(results)


In [5]:
Data_path=os.path.join('Mp_data')

actions=np.array(['hello','iloveyou','thanks'])

no_of_sequence=30

sequence_length=30



In [35]:
for action in actions:
    for s in range(no_of_sequence):
        try:
            os.makedirs(os.path.join(Data_path,action,str(s)))
        except:
            pass

In [37]:
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  for action in actions:
      for sequence in range(no_of_sequence):
          for frames in range(30):
 
 
            ret,frame=cap.read()
    
 
            image,results=holistic_detection(frame,holistic)
            draw_styled_landmarks(image, results)
            if frames == 0:
              cv2.putText(image,"Strating... ",(150,120),cv2.FONT_HERSHEY_SIMPLEX,1,(10,100,255),1,cv2.LINE_AA)
              cv2.putText(image,"Collecting frames for {} video number {}".format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
              cv2.imshow('OpenCv Feed',image)
              cv2.waitKey(2000)
            else: 
              cv2.putText(image,"Collecting frames for {} video number {}".format(action,sequence),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
              cv2.imshow('OpenCv Feed',image)
         
            keypoints=extract_keypoints(results)
            npy_path=os.path.join(Data_path,action,str(sequence),str(frames))
            np.save(npy_path,keypoints)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
  cap.release
  cv2.destroyAllWindows()  

In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical



In [7]:
label_map={'hello':0,'iloveyou':1,'thanks':2}

actions=np.array(['hello','iloveyou','thanks'])

In [66]:
sequences,labels=[],[]
for action in actions:
    for sequence in range(sequence_length):
        window=[]
        for frame_num in range(30):
            res=np.load(os.path.join(Data_path,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)   
        labels.append(label_map[action])

In [67]:
X=np.array(sequences)
y=to_categorical(labels).astype(int)

In [68]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.05)

Training and Testing

In [15]:
from tensorflow.keras.models import Sequential
from  tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [26]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [69]:
model=Sequential()
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662)))
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))




In [71]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [79]:
model.fit(X_train,y_train,epochs=800,callbacks=[tb_callback])

Epoch 1/800
3/3 [==============================] - 2s 126ms/step - loss: 0.5483 - categorical_accuracy: 0.7412
Epoch 2/800
3/3 [==============================] - 0s 122ms/step - loss: 0.4625 - categorical_accuracy: 0.7647
Epoch 3/800
3/3 [==============================] - 0s 124ms/step - loss: 0.4370 - categorical_accuracy: 0.7765
Epoch 4/800
3/3 [==============================] - 0s 127ms/step - loss: 0.5888 - categorical_accuracy: 0.6824
Epoch 5/800
3/3 [==============================] - 0s 129ms/step - loss: 0.5580 - categorical_accuracy: 0.6471
Epoch 6/800
3/3 [==============================] - 0s 129ms/step - loss: 0.4264 - categorical_accuracy: 0.7647
Epoch 7/800
3/3 [==============================] - 0s 125ms/step - loss: 0.6101 - categorical_accuracy: 0.6941
Epoch 8/800
3/3 [==============================] - 0s 128ms/step - loss: 0.5365 - categorical_accuracy: 0.7882
Epoch 9/800
3/3 [==============================] - 0s 126ms/step - loss: 0.6402 - categorical_accuracy: 0.6471
E

KeyboardInterrupt: 

In [80]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 30, 64)            442112    
                                                                 
 lstm_16 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_17 (LSTM)              (None, 64)                49408     
                                                                 
 dense_12 (Dense)            (None, 64)                4160      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 3)                 99        
                                                                 
Total params: 596675 (2.28 MB)
Trainable params: 59667

In [81]:




from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [74]:
actions[np.argmax(res[4])]

'hello'

In [83]:

actions[np.argmax(y_test[4])]

'iloveyou'

In [82]:

yhat = model.predict(X_test)

ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)



1/1 [==============================] - 0s 68ms/step


array([[[3, 0],
        [0, 2]],

       [[2, 0],
        [1, 2]],

       [[4, 1],
        [0, 0]]], dtype=int64)

In [84]:
accuracy_score(ytrue, yhat)


0.8

In [64]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [50]:
import matplotlib.pyplot  as plt

In [85]:
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = holistic_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

KeyboardInterrupt: 

: 

In [58]:
cap.release()
cv2.destroyAllWindows()